In [5]:
!pip install -q selenium webdriver-manager pandas

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from io import StringIO

# Настройка Selenium WebDriver
def create_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Запуск без интерфейса браузера
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# Функция для загрузки страницы с тремя попытками
def fetch_html(driver, page_no, retries=3):
    url = f"https://declaration.rostrud.gov.ru/declaration/index?page={page_no}&per-page=50"
    print(f"Загружаем страницу {page_no}: {url}")

    for attempt in range(retries):
        try:
            driver.get(url)
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.ID, "reestr"))
            )
            return driver.page_source
        except Exception as e:
            print(f"Попытка {attempt + 1} не удалась для страницы {page_no}: {e}")
            time.sleep(2)  # Пауза перед повторной попыткой

    print(f"Ошибка при загрузке страницы {page_no} после {retries} попыток.")
    return None

# # Парсинг HTML-страницы
# def parse_html(html_content):
#     try:
#         df = pd.read_html(html_content, attrs={"id": "reestr"})[0]
#         df.columns = df.iloc[0]
#         df = df[1:].reset_index(drop=True)
#         return df
#     except ValueError:
#         print("Таблица с указанным ID не найдена.")
#         return pd.DataFrame()

# Парсинг HTML-страницы с использованием StringIO
def parse_html(html_content):
    try:
        # Оборачиваем html_content в StringIO перед передачей в read_html
        df = pd.read_html(StringIO(html_content), attrs={"id": "reestr"})[0]
        df.columns = df.iloc[0]  # Первая строка становится заголовком
        df = df[1:].reset_index(drop=True)  # Убираем старую первую строку
        return df
    except ValueError:
        print("Таблица с указанным ID не найдена.")
        return pd.DataFrame()
# Сохранение данных в CSV
def save_data(data, part):
    filename = f"reestr_data_part_{part}.csv"
    data.to_csv(filename, index=False, encoding="utf-8")
    print(f"Данные сохранены в {filename}")

# Основная функция
def main():
    driver = create_driver()
    part = 1
    all_data = pd.DataFrame()
    save_interval = 100  # Сохраняем каждые 50 страниц
    total_pages = 28045  # Всего страниц

    try:
        # for page_no in range(1, total_pages + 1):
        for page_no in range(1, total_pages + 1):
            html_content = fetch_html(driver, page_no)

            if html_content:
                df = parse_html(html_content)
                if not df.empty:
                    all_data = pd.concat([all_data, df], ignore_index=True)
                else:
                    print(f"Пустая таблица на странице {page_no}.")
            else:
                print(f"Ошибка при загрузке страницы {page_no}")

            # Сохраняем каждые 100 страниц
            if page_no % save_interval == 0 or page_no == total_pages:
                print(f"Сохраняем данные после страницы {page_no}...")
                save_data(all_data, part)
                all_data = pd.DataFrame()  # Очищаем данные для следующей части
                part += 1

            time.sleep(2)  # Задержка между запросами

    finally:
        driver.quit()
        print("Выгрузка данных завершена.")

if __name__ == "__main__":
    main()


Загружаем страницу 3762: https://declaration.rostrud.gov.ru/declaration/index?page=3762&per-page=50
Загружаем страницу 3763: https://declaration.rostrud.gov.ru/declaration/index?page=3763&per-page=50
Загружаем страницу 3764: https://declaration.rostrud.gov.ru/declaration/index?page=3764&per-page=50
Загружаем страницу 3765: https://declaration.rostrud.gov.ru/declaration/index?page=3765&per-page=50
Загружаем страницу 3766: https://declaration.rostrud.gov.ru/declaration/index?page=3766&per-page=50
Загружаем страницу 3767: https://declaration.rostrud.gov.ru/declaration/index?page=3767&per-page=50
Загружаем страницу 3768: https://declaration.rostrud.gov.ru/declaration/index?page=3768&per-page=50
Загружаем страницу 3769: https://declaration.rostrud.gov.ru/declaration/index?page=3769&per-page=50
Загружаем страницу 3770: https://declaration.rostrud.gov.ru/declaration/index?page=3770&per-page=50
Загружаем страницу 3771: https://declaration.rostrud.gov.ru/declaration/index?page=3771&per-page=50


In [2]:
import pandas as pd
import glob

def load_and_combine_csv(file_pattern="reestr_data_part_*.csv"):
    # Создаём список файлов, соответствующих шаблону
    files = glob.glob(f"{file_pattern}")

    print(f"Найдено файлов: {len(files)}")

    # Загружаем и объединяем все файлы в один DataFrame
    dataframes = []
    for file in files:
        print(f"Загружаем файл: {file}")
        df = pd.read_csv(file, encoding='utf-8')
        dataframes.append(df)

    # Объединяем все DataFrame в один
    combined_df = pd.concat(dataframes, ignore_index=True)

    print(f"Общий размер объединённого DataFrame: {combined_df.shape}")
    return combined_df

def main():

    # Загружаем и объединяем все части
    combined_df = load_and_combine_csv()

    # Сохраняем объединённые данные в итоговый CSV
    combined_df.to_csv(f"reestr_full_data.csv", index=False, encoding='utf-8')
    print("Объединённый файл сохранён как reestr_full_data.csv")

if __name__ == "__main__":
    main()


Найдено файлов: 67
Загружаем файл: reestr_data_part_1.csv
Загружаем файл: reestr_data_part_10.csv
Загружаем файл: reestr_data_part_11.csv
Загружаем файл: reestr_data_part_12.csv
Загружаем файл: reestr_data_part_13.csv
Загружаем файл: reestr_data_part_14.csv
Загружаем файл: reestr_data_part_15.csv
Загружаем файл: reestr_data_part_16.csv
Загружаем файл: reestr_data_part_17.csv
Загружаем файл: reestr_data_part_18.csv
Загружаем файл: reestr_data_part_19.csv
Загружаем файл: reestr_data_part_2.csv
Загружаем файл: reestr_data_part_20.csv
Загружаем файл: reestr_data_part_21.csv
Загружаем файл: reestr_data_part_22.csv
Загружаем файл: reestr_data_part_23.csv
Загружаем файл: reestr_data_part_24.csv
Загружаем файл: reestr_data_part_25.csv
Загружаем файл: reestr_data_part_26.csv
Загружаем файл: reestr_data_part_27.csv
Загружаем файл: reestr_data_part_28.csv
Загружаем файл: reestr_data_part_29.csv
Загружаем файл: reestr_data_part_3.csv
Загружаем файл: reestr_data_part_30.csv
Загружаем файл: reestr_d